## Copyright 2022 Google LLC. Double-click for license information.

In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Prompt-to-Prompt with Stable Diffusion

In [ ]:
from typing import Optional, Union, Tuple, List, Callable, Dict
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler
import torch.nn.functional as nnf
import numpy as np
import abc
import ptp_utils
import seq_aligner
from PIL import Image

For loading the Stable Diffusion using Diffusers, follow the instuctions https://huggingface.co/blog/stable_diffusion and update ```MY_TOKEN``` with your token.
Set ```LOW_RESOURCE``` to ```True``` for running on 12GB GPU.

In [ ]:
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
# MY_TOKEN = ''
LOW_RESOURCE = False 
NUM_DIFFUSION_STEPS = 50
GUIDANCE_SCALE = 7.5
MAX_NUM_WORDS = 77
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
ldm_stable = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", local_files_only=True, scheduler=scheduler).to(device)
try:
    ldm_stable.disable_xformers_memory_efficient_attention()
except AttributeError:
    print("Attribute disable_xformers_memory_efficient_attention() is missing")
tokenizer = ldm_stable.tokenizer

## Prompt-to-Prompt Attnetion Controllers
Our main logic is implemented in the `forward` call in an `AttentionControl` object.
The forward is called in each attention layer of the diffusion model and it can modify the input attnetion weights `attn`.

`is_cross`, `place_in_unet in ("down", "mid", "up")`, `AttentionControl.cur_step` help us track the exact attention layer and timestamp during the diffusion iference.


In [ ]:
class LocalBlend:

    def __call__(self, x_t, attention_store):
        k = 1
        maps = attention_store["down_cross"][2:4] + attention_store["up_cross"][:3]
        maps = [item.reshape(self.alpha_layers.shape[0], -1, 1, 16, 16, MAX_NUM_WORDS) for item in maps]
        maps = torch.cat(maps, dim=1)
        maps = (maps * self.alpha_layers).sum(-1).mean(1)
        mask = nnf.max_pool2d(maps, (k * 2 + 1, k * 2 +1), (1, 1), padding=(k, k))
        mask = nnf.interpolate(mask, size=(x_t.shape[2:]))
        mask = mask / mask.max(2, keepdims=True)[0].max(3, keepdims=True)[0]
        mask = mask.gt(self.threshold)
        mask = (mask[:1] + mask[1:]).float()
        x_t = x_t[:1] + mask * (x_t - x_t[:1])
        return x_t
       
    def __init__(self, prompts: List[str], words: [List[List[str]]], threshold=.3):
        alpha_layers = torch.zeros(len(prompts),  1, 1, 1, 1, MAX_NUM_WORDS)
        for i, (prompt, words_) in enumerate(zip(prompts, words)):
            if type(words_) is str:
                words_ = [words_]
            for word in words_:
                ind = ptp_utils.get_word_inds(prompt, word, tokenizer)
                alpha_layers[i, :, :, :, :, ind] = 1
        self.alpha_layers = alpha_layers.to(device)
        self.threshold = threshold


class AttentionControl(abc.ABC):
    
    def step_callback(self, x_t):
        return x_t
    
    def between_steps(self):
        return
    
    @property
    def num_uncond_att_layers(self):
        return self.num_att_layers if LOW_RESOURCE else 0
    
    @abc.abstractmethod
    def forward (self, attn, is_cross: bool, place_in_unet: str):
        raise NotImplementedError

    def __call__(self, attn, is_cross: bool, place_in_unet: str):
        if self.cur_att_layer >= self.num_uncond_att_layers:
            if LOW_RESOURCE:
                attn = self.forward(attn, is_cross, place_in_unet)
            else:
                h = attn.shape[0]
                attn[h // 2:] = self.forward(attn[h // 2:], is_cross, place_in_unet)
        self.cur_att_layer += 1
        if self.cur_att_layer == self.num_att_layers + self.num_uncond_att_layers:
            self.cur_att_layer = 0
            self.cur_step += 1
            self.between_steps()
        return attn
    
    def reset(self):
        self.cur_step = 0
        self.cur_att_layer = 0

    def __init__(self):
        self.cur_step = 0
        self.num_att_layers = -1
        self.cur_att_layer = 0

class EmptyControl(AttentionControl):
    
    def forward (self, attn, is_cross: bool, place_in_unet: str):
        return attn
    
    
class AttentionStore(AttentionControl):

    @staticmethod
    def get_empty_store():
        return {"down_cross": [], "mid_cross": [], "up_cross": [],
                "down_self": [],  "mid_self": [],  "up_self": []}

    def forward(self, attn, is_cross: bool, place_in_unet: str):
        key = f"{place_in_unet}_{'cross' if is_cross else 'self'}"
        if attn.shape[1] <= 32 ** 2:  # avoid memory overhead
            self.step_store[key].append(attn)
        return attn

    def between_steps(self):
        if len(self.attention_store) == 0:
            self.attention_store = self.step_store
        else:
            for key in self.attention_store:
                for i in range(len(self.attention_store[key])):
                    self.attention_store[key][i] += self.step_store[key][i]
        self.step_store = self.get_empty_store()

    def get_average_attention(self):
        average_attention = {key: [item / self.cur_step for item in self.attention_store[key]] for key in self.attention_store}
        return average_attention


    def reset(self):
        super(AttentionStore, self).reset()
        self.step_store = self.get_empty_store()
        self.attention_store = {}

    def __init__(self):
        super(AttentionStore, self).__init__()
        self.step_store = self.get_empty_store()
        self.attention_store = {}

        
class AttentionControlEdit(AttentionStore, abc.ABC):
    
    def step_callback(self, x_t):
        if self.local_blend is not None:
            x_t = self.local_blend(x_t, self.attention_store)
        return x_t
        
    def replace_self_attention(self, attn_base, att_replace):
        if att_replace.shape[2] <= 16 ** 2:
            return attn_base.unsqueeze(0).expand(att_replace.shape[0], *attn_base.shape)
        else:
            return att_replace
    
    @abc.abstractmethod
    def replace_cross_attention(self, attn_base, att_replace):
        raise NotImplementedError
    
    def forward(self, attn, is_cross: bool, place_in_unet: str):
        super(AttentionControlEdit, self).forward(attn, is_cross, place_in_unet)
        if is_cross or (self.num_self_replace[0] <= self.cur_step < self.num_self_replace[1]):
            h = attn.shape[0] // (self.batch_size)
            attn = attn.reshape(self.batch_size, h, *attn.shape[1:])
            attn_base, attn_repalce = attn[0], attn[1:]
            if is_cross:
                alpha_words = self.cross_replace_alpha[self.cur_step]
                attn_repalce_new = self.replace_cross_attention(attn_base, attn_repalce) * alpha_words + (1 - alpha_words) * attn_repalce
                attn[1:] = attn_repalce_new
            else:
                attn[1:] = self.replace_self_attention(attn_base, attn_repalce)
            attn = attn.reshape(self.batch_size * h, *attn.shape[2:])
        return attn
    
    def __init__(self, prompts, num_steps: int,
                 cross_replace_steps: Union[float, Tuple[float, float], Dict[str, Tuple[float, float]]],
                 self_replace_steps: Union[float, Tuple[float, float]],
                 local_blend: Optional[LocalBlend]):
        super(AttentionControlEdit, self).__init__()
        self.batch_size = len(prompts)
        self.cross_replace_alpha = ptp_utils.get_time_words_attention_alpha(prompts, num_steps, cross_replace_steps, tokenizer).to(device)
        if type(self_replace_steps) is float:
            self_replace_steps = 0, self_replace_steps
        self.num_self_replace = int(num_steps * self_replace_steps[0]), int(num_steps * self_replace_steps[1])
        self.local_blend = local_blend

class AttentionReplace(AttentionControlEdit):

    def replace_cross_attention(self, attn_base, att_replace):
        return torch.einsum('hpw,bwn->bhpn', attn_base, self.mapper)
      
    def __init__(self, prompts, num_steps: int, cross_replace_steps: float, self_replace_steps: float,
                 local_blend: Optional[LocalBlend] = None):
        super(AttentionReplace, self).__init__(prompts, num_steps, cross_replace_steps, self_replace_steps, local_blend)
        self.mapper = seq_aligner.get_replacement_mapper(prompts, tokenizer).to(device)
        

class AttentionRefine(AttentionControlEdit):

    def replace_cross_attention(self, attn_base, att_replace):
        attn_base_replace = attn_base[:, :, self.mapper].permute(2, 0, 1, 3)
        attn_replace = attn_base_replace * self.alphas + att_replace * (1 - self.alphas)
        return attn_replace

    def __init__(self, prompts, num_steps: int, cross_replace_steps: float, self_replace_steps: float,
                 local_blend: Optional[LocalBlend] = None):
        super(AttentionRefine, self).__init__(prompts, num_steps, cross_replace_steps, self_replace_steps, local_blend)
        self.mapper, alphas = seq_aligner.get_refinement_mapper(prompts, tokenizer)
        self.mapper, alphas = self.mapper.to(device), alphas.to(device)
        self.alphas = alphas.reshape(alphas.shape[0], 1, 1, alphas.shape[1])

class AttentionReweight(AttentionControlEdit):

    def replace_cross_attention(self, attn_base, att_replace):
        if self.prev_controller is not None:
            attn_base = self.prev_controller.replace_cross_attention(attn_base, att_replace)
        attn_replace = attn_base[None, :, :, :] * self.equalizer[:, None, None, :]
        return attn_replace

    def __init__(self, prompts, num_steps: int, cross_replace_steps: float, self_replace_steps: float, equalizer,
                local_blend: Optional[LocalBlend] = None, controller: Optional[AttentionControlEdit] = None):
        super(AttentionReweight, self).__init__(prompts, num_steps, cross_replace_steps, self_replace_steps, local_blend)
        self.equalizer = equalizer.to(device)
        self.prev_controller = controller


def get_equalizer(text: str, word_select: Union[int, Tuple[int, ...]], values: Union[List[float],
                  Tuple[float, ...]]):
    if type(word_select) is int or type(word_select) is str:
        word_select = (word_select,)
    equalizer = torch.ones(len(values), 77)
    values = torch.tensor(values, dtype=torch.float32)
    for word in word_select:
        inds = ptp_utils.get_word_inds(text, word, tokenizer)
        equalizer[:, inds] = values
    return equalizer

def aggregate_attention(attention_store: AttentionStore, res: int, from_where: List[str], is_cross: bool, select: int):
    out = []
    attention_maps = attention_store.get_average_attention()
    num_pixels = res ** 2
    for location in from_where:
        for item in attention_maps[f"{location}_{'cross' if is_cross else 'self'}"]:
            if item.shape[1] == num_pixels:
                cross_maps = item.reshape(len(prompts), -1, res, res, item.shape[-1])[select]
                out.append(cross_maps)
    out = torch.cat(out, dim=0)
    out = out.sum(0) / out.shape[0]
    return out.cpu()


def show_cross_attention(attention_store: AttentionStore, res: int, from_where: List[str], select: int = 0):
    tokens = tokenizer.encode(prompts[select])
    decoder = tokenizer.decode
    attention_maps = aggregate_attention(attention_store, res, from_where, True, select)
    images = []
    for i in range(len(tokens)):
        image = attention_maps[:, :, i]
        image = 255 * image / image.max()
        image = image.unsqueeze(-1).expand(*image.shape, 3)
        image = image.numpy().astype(np.uint8)
        image = np.array(Image.fromarray(image).resize((256, 256)))
        image = ptp_utils.text_under_image(image, decoder(int(tokens[i])))
        images.append(image)
    ptp_utils.view_images(np.stack(images, axis=0))
    

def show_self_attention_comp(attention_store: AttentionStore, res: int, from_where: List[str],
                        max_com=10, select: int = 0):
    attention_maps = aggregate_attention(attention_store, res, from_where, False, select).numpy().reshape((res ** 2, res ** 2))
    u, s, vh = np.linalg.svd(attention_maps - np.mean(attention_maps, axis=1, keepdims=True))
    images = []
    for i in range(max_com):
        image = vh[i].reshape(res, res)
        image = image - image.min()
        image = 255 * image / image.max()
        image = np.repeat(np.expand_dims(image, axis=2), 3, axis=2).astype(np.uint8)
        image = Image.fromarray(image).resize((256, 256))
        image = np.array(image)
        images.append(image)
    ptp_utils.view_images(np.concatenate(images, axis=1))

In [ ]:
def load_512(image_path, left=0, right=0, top=0, bottom=0):
    if type(image_path) is str:
        image = np.array(Image.open(image_path))[:, :, :3]
    else:
        image = image_path
    h, w, c = image.shape
    left = min(left, w-1)
    right = min(right, w - left - 1)
    top = min(top, h - left - 1)
    bottom = min(bottom, h - top - 1)
    image = image[top:h-bottom, left:w-right]
    h, w, c = image.shape
    if h < w:
        offset = (w - h) // 2
        image = image[:, offset:offset + h]
    elif w < h:
        offset = (h - w) // 2
        image = image[offset:offset + w]
    image = np.array(Image.fromarray(image).resize((512, 512)))
    return image

In [ ]:
class ERDDCIInversion:
    
    def prev_step(self, model_output: Union[torch.FloatTensor, np.ndarray], timestep: int, sample: Union[torch.FloatTensor, np.ndarray]):
        prev_timestep = timestep - self.scheduler.config.num_train_timesteps // self.scheduler.num_inference_steps
        alpha_prod_t = self.scheduler.alphas_cumprod[timestep]
        alpha_prod_t_prev = self.scheduler.alphas_cumprod[prev_timestep] if prev_timestep >= 0 else self.scheduler.final_alpha_cumprod
        beta_prod_t = 1 - alpha_prod_t
        pred_original_sample = (sample - beta_prod_t ** 0.5 * model_output) / alpha_prod_t ** 0.5
        pred_sample_direction = (1 - alpha_prod_t_prev) ** 0.5 * model_output
        prev_sample = alpha_prod_t_prev ** 0.5 * pred_original_sample + pred_sample_direction
        return prev_sample
    
    def next_step(self, model_output: Union[torch.FloatTensor, np.ndarray], timestep: int, sample: Union[torch.FloatTensor, np.ndarray], ori_latent):
        timestep, next_timestep = min(timestep - self.scheduler.config.num_train_timesteps // self.scheduler.num_inference_steps, 999), timestep
        alpha_prod_t = self.scheduler.alphas_cumprod[timestep] if timestep >= 0 else self.scheduler.final_alpha_cumprod
        alpha_prod_t_next = self.scheduler.alphas_cumprod[next_timestep]
        beta_prod_t = 1 - alpha_prod_t
        next_original_sample = (sample - beta_prod_t ** 0.5 * model_output) / alpha_prod_t ** 0.5
        original_sample = ori_latent
        next_sample_direction = (1 - alpha_prod_t_next) ** 0.5 * model_output
        next_sample_direction_1 = (1 - alpha_prod_t_next) ** 0.5 * torch.randn_like(model_output)
        next_sample = alpha_prod_t_next ** 0.5 * next_original_sample + next_sample_direction
        return next_sample
    
    def get_noise_pred_single(self, latents, t, context):
        noise_pred = self.model.unet(latents, t, encoder_hidden_states=context)["sample"]
        return noise_pred

    # def get_noise_pred(self, latents, t, is_forward=True, context=None):
    #     latents_input = torch.cat([latents] * 2)
    #     if context is None:
    #         context = self.context
    #     guidance_scale = 1 if is_forward else GUIDANCE_SCALE
    #     noise_pred = self.model.unet(latents_input, t, encoder_hidden_states=context)["sample"]
    #     noise_pred_uncond, noise_prediction_text = noise_pred.chunk(2)
    #     noise_pred = noise_pred_uncond + guidance_scale * (noise_prediction_text - noise_pred_uncond)
    #     if is_forward:
    #         latents = self.next_step(noise_pred, t, latents)
    #     else:
    #         latents = self.prev_step(noise_pred, t, latents)
    #     return latents
    

    @torch.no_grad()
    def latent2image(self, latents, return_type='np'):
        latents = 1 / 0.18215 * latents.detach()
        image = self.model.vae.decode(latents)['sample']
        if return_type == 'np':
            image = (image / 2 + 0.5).clamp(0, 1)
            image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
            image = (image * 255).astype(np.uint8)
        return image

    @torch.no_grad()
    def image2latent(self, image):
        with torch.no_grad():
            if type(image) is Image:
                image = np.array(image)
            if type(image) is torch.Tensor and image.dim() == 4:
                latents = image
            else:
                image = torch.from_numpy(image).float() / 127.5 - 1
                image = image.permute(2, 0, 1).unsqueeze(0).to(device)
                latents = self.model.vae.encode(image)['latent_dist'].mean
                latents = latents * 0.18215
        return latents

    @torch.no_grad()
    def init_prompt(self, prompt: str):
        uncond_input = self.model.tokenizer(
            [""], padding="max_length", max_length=self.model.tokenizer.model_max_length,
            return_tensors="pt"
        )
        uncond_embeddings = self.model.text_encoder(uncond_input.input_ids.to(self.model.device))[0]
        text_input = self.model.tokenizer(
            [prompt],
            padding="max_length",
            max_length=self.model.tokenizer.model_max_length,
            truncation=True,
            return_tensors="pt",
        )
        text_embeddings = self.model.text_encoder(text_input.input_ids.to(self.model.device))[0]
        self.context = torch.cat([uncond_embeddings, text_embeddings])
        self.prompt = prompt

    @torch.no_grad()
    def ddim_loop(self, latent):
        uncond_embeddings, cond_embeddings = self.context.chunk(2)
        ori_latent = latent
        aux_latent = latent
        aux_all_latent = [aux_latent]
        ori_all_latent = [ori_latent]
        for i in range(NUM_DIFFUSION_STEPS):
            t = self.model.scheduler.timesteps[len(self.model.scheduler.timesteps) - i - 1]
            
            noise_pred = self.get_noise_pred_single(ori_latent, t,  cond_embeddings)
            ori_latent = self.next_step(noise_pred, t, t, ori_latent)
            ori_all_latent.append(ori_latent)
            
            noise_pred = self.get_noise_pred_single(ori_latent, t,  cond_embeddings)
            aux_latent = self.next_step(noise_pred, t, aux_latent)
            aux_all_latent.append(aux_latent)
        return aux_all_latent, ori_all_latent

    @property
    def scheduler(self):
        return self.model.scheduler

    @torch.no_grad()
    def ddim_inversion(self, image):
        latent = self.image2latent(image)
        image_rec = self.latent2image(latent)
        aux_ddim_latents, ori_ddim_latents = self.ddim_loop(latent)
        return image_rec, aux_ddim_latents, ori_ddim_latents

    
    def invert(self, image_path, prompt: str, offsets=(0,0,0,0), num_inner_steps=10, early_stop_epsilon=1e-5, verbose=False):
        self.init_prompt(prompt)
        ptp_utils.register_attention_control(self.model, None)
        image_gt = load_512(image_path)
        if verbose:
            print("ERDDCI inversion...")
        print('image_gt',image_gt)
        image_rec, aux_ddim_latents, ori_ddim_latents = self.ddim_inversion(image_gt)
        return (image_gt, image_rec), aux_ddim_latents, ori_ddim_latents
        
    def __init__(self, model):
        scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False,
                                  set_alpha_to_one=False)
        self.model = model
        self.tokenizer = self.model.tokenizer
        self.model.scheduler.set_timesteps(NUM_DIFFUSION_STEPS)
        self.prompt = None
        self.context = None

erddci_inversion = ERDDCIInversion(ldm_stable)

In [ ]:
# TODO: need rewrite
def diffusion_step(model, controller, latents, context, t, guidance_scale, low_resource=False):
    if low_resource:
        noise_pred_uncond = model.unet(latents, t, encoder_hidden_states=context[0])["sample"]
        noise_prediction_text = model.unet(latents, t, encoder_hidden_states=context[1])["sample"]
    else:
        latents_input = torch.cat([latents] * 2)
        noise_pred = model.unet(latents_input, t, encoder_hidden_states=context)["sample"]
        noise_pred_uncond, noise_prediction_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_prediction_text - noise_pred_uncond)
    latents = model.scheduler.step(noise_pred, t, latents)["prev_sample"]
    latents = controller.step_callback(latents)
    return latents

@torch.no_grad()
def text2image_ldm_stable(
    model,
    prompt:  List[str],
    controller,
    num_inference_steps: int = 50,
    guidance_scale: Optional[float] = 7.5,
    generator: Optional[torch.Generator] = None,
    latent: Optional[torch.FloatTensor] = None,
    uncond_embeddings=None,
    start_time=50,
    return_type='image'
):
    batch_size = len(prompt)
    ptp_utils.register_attention_control(model, controller)
    height = width = 512
    
    text_input = model.tokenizer(
        prompt,
        padding="max_length",
        max_length=model.tokenizer.model_max_length,
        truncation=True,
        return_tensors="pt",
    )
    text_embeddings = model.text_encoder(text_input.input_ids.to(model.device))[0]
    max_length = text_input.input_ids.shape[-1]
    if uncond_embeddings is None:
        uncond_input = model.tokenizer(
            [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
        )
        uncond_embeddings_ = model.text_encoder(uncond_input.input_ids.to(model.device))[0]
    else:
        uncond_embeddings_ = None

    latent, latents = ptp_utils.init_latent(latent, model, height, width, generator, batch_size)
    model.scheduler.set_timesteps(num_inference_steps)
    for i, t in enumerate(tqdm(model.scheduler.timesteps[-start_time:])):
        if uncond_embeddings_ is None:
            context = torch.cat([uncond_embeddings[i].expand(*text_embeddings.shape), text_embeddings])
        else:
            context = torch.cat([uncond_embeddings_, text_embeddings])
        latents = ptp_utils.diffusion_step(model, controller, latents, context, t, guidance_scale, low_resource=False)
        
    if return_type == 'image':
        image = ptp_utils.latent2image(model.vae, latents)
    else:
        image = latents
    return image, latent

def run_and_display(prompts, controller, latent=None, run_baseline=False, generator=None):
    if run_baseline:
        print("w.o. prompt-to-prompt")
        images, latent = run_and_display(prompts, EmptyControl(), latent=latent, run_baseline=False, generator=generator)
        print("with prompt-to-prompt")
    images, x_t = ptp_utils.text2image_ldm_stable(ldm_stable, prompts, controller, latent=latent, num_inference_steps=NUM_DIFFUSION_STEPS, guidance_scale=3, generator=generator, low_resource=LOW_RESOURCE)
    ptp_utils.view_images(images)
    return images, x_t

In [ ]:
def run_and_display_edit(prompts, controller, latent=None, latent_unt=None, guidance_scale=1.0, guidance_start_time = 1, run_baseline=False, generator=None):
    if run_baseline:
        print("w.o. prompt-to-prompt")
        images, latent = run_and_display_edit(prompts, EmptyControl(), latent=latent, latent_unt=latent_unt, 
                    guidance_scale=guidance_scale, guidance_start_time = guidance_start_time, run_baseline=False, generator=generator)
        print("with prompt-to-prompt")
    images, x_t = ptp_utils.text2image_ldm_stable_edit(ldm_stable, prompts, controller, latent=latent, latent_unt=latent_unt, num_inference_steps=NUM_DIFFUSION_STEPS, guidance_scale=guidance_scale, guidance_start_time=guidance_start_time, generator=generator, low_resource=LOW_RESOURCE)
    ptp_utils.view_images(images)
    return images, x_t

## Cross-Attention Visualization
First let's generate an image and visualize the cross-attention maps for each word in the prompt.
Notice, we normalize each map to 0-1.

In [ ]:
from PIL import Image
import os
from IPython.display import display, Markdown

# 设置描述文件和图像文件夹的路径
token_file_path = 'flickr30k_dataset/results_20130124.token'
image_folder_path = 'flickr30k_dataset/flickr30k-images/'

# 解析描述文件
def parse_captions(file_path, target_caption_number="3"):
    captions_dict = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split('\t')
            if len(parts) < 2:  # 确保行内有足够的分割元素
                continue
            caption_id = parts[0]
            caption_text = parts[1]
            image_id, caption_num = caption_id.split('#')
            if caption_num == target_caption_number:
                captions_dict[image_id] = caption_text
    return captions_dict

# 加载图像并返回图像对象和描述
def get_image_and_captions(image_path, captions_dict):
    # 获取图像ID
    image_id = os.path.basename(image_path).split('#')[0]
    print(f"Checking captions for image ID: {image_id}")

    # 加载图像
    try:
        img = Image.open(image_path)
    except IOError:
        print(f"Error opening image {image_path}")
        return None, ["Error opening image."]

    # 获取图像描述
    if image_id in captions_dict:
        return img, image_id, captions_dict[image_id]
    else:
        return img, ["No captions found for this image."]

# 主程序
if __name__ == "__main__":
    # 解析描述文件
    captions_dict = parse_captions(token_file_path)


    # 设置变量k为第k张图像
    k = 12447  # 例如，抽取第10张图像

    # 获取图像文件列表
    images = [img for img in os.listdir(image_folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # 检查k是否在有效范围内
    if k < 1 or k > len(images):
        print("Error: 'k' is out of the valid range.")
    else:
        # 抽取第k张图像
        image_path = os.path.join(image_folder_path, '2833560457.jpg')  # 数组索引从0开始，所以k-1
        print("qq:",images[k-1])
        image, image_id, captions = get_image_and_captions(image_path, captions_dict)
        
        # 在Jupyter Notebook中显示图像和描述
        if image:
            display(image)
            print(captions)

In [ ]:
import matplotlib.pyplot as plt
def save_plot(image_data, filename):
    plt.figure()
    plt.imshow(image_data)
    plt.axis('off')  # 不显示坐标轴
    plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
# image_path = f"flickr30k_dataset/flickr30k-images/{image_id}"
# prompt = captions
image_path = "./example_images/a2dcdb1105ffdbf66696d6465c6072c.jpg"
prompt ="A blue SUV in the woods"
(image_gt, image_enc), ddim_latents, ori_ddim_latents = dc_inversion.invert(image_path, prompt, offsets=(0,0,200,0), verbose=True)
save_plot(image_gt, 'images/image6_0.png')
print("Modify or remove offsets according to your image!")

In [ ]:
g_cpu = torch.Generator().manual_seed(8888)
# prompts = [captions]
prompts = ["A blue SUV in the woods"]
controller = AttentionStore()
image, _ = run_and_display_edit(prompts, controller, latent=ddim_latents[-1], latent_unt=ori_ddim_latents, guidance_scale=1.0, guidance_start_time = 0.0, run_baseline=False, generator=g_cpu)
show_cross_attention(controller, res=16, from_where=("up", "down"))

In [ ]:
# prompts = [prompt]
# controller = AttentionStore()
# image_inv, _ = run_and_display_edit(prompts, controller, latent=ori_ddim_latents[-1], latent_unt=ori_ddim_latents, guidance_scale=1.0, guidance_start_time = 0.0, run_baseline=False, generator=g_cpu)
# show_cross_attention(controller, res=16, from_where=("up", "down"))
# print("showing from left to right: the ground truth image, the vq-autoencoder reconstruction, the null-text inverted image")
# ptp_utils.view_images([image_gt, image_enc, image_inv[0]])

In [ ]:
# g_cpu = torch.Generator().manual_seed(8888)
# prompts = ["a tiger sitting next to a mirror"]
# controller = AttentionStore()
# image, x_t = run_and_display(prompts, controller, latent=ori_ddim_latents[-1], run_baseline=False, generator=g_cpu)
# show_cross_attention(controller, res=16, from_where=("up", "down"))

## Replacement edit

In [ ]:
prompts = ["A blue SUV in the woods",
           "A blue SUV in the city"]
lb = LocalBlend(prompts, ("woods", "city"))
controller = AttentionReplace(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.2, self_replace_steps=0.5, local_blend=lb)
image_inv, _ = run_and_display_edit(prompts, controller, latent=ddim_latents[-1], latent_unt=ori_ddim_latents, guidance_scale=4.5, guidance_start_time = 0.1, run_baseline=False, generator=g_cpu)
save_plot(image_inv[1], 'images/image6_4.png')

In [ ]:
prompts = ["A dragonfly landed on a branch",
           "A dragonfly landed on a flower"]
lb = LocalBlend(prompts, ("branch", "flower"))
controller = AttentionReplace(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.3, self_replace_steps=0.4, local_blend=lb)
image_inv, _ = run_and_display_edit(prompts, controller, latent=ddim_latents[-1], latent_unt=ori_ddim_latents, guidance_scale=6, guidance_start_time = 0.2, run_baseline=False, generator=g_cpu)
# save_plot(image_inv[1], 'images/image2_1.png')


### Modify Cross-Attention injection #steps for specific words
Next, we can reduce the restriction on our lion by reducing the number of cross-attention injection with respect to the word "lion".

In [ ]:
prompts = ["a cat sitting next to a mirror",
           "a tiger sitting next to a mirror"]
controller = AttentionReplace(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps={"default_": 1., "tiger": .8},
                              self_replace_steps=0.6)
image_inv, _ = run_and_display_edit(prompts, controller, latent=ddim_latents[-1], latent_unt=ori_ddim_latents, guidance_scale=8, guidance_start_time = 0.3, run_baseline=True, generator=g_cpu)

### Local Edit
Lastly, if we want to preseve the original burger, we can apply a local edit with respect to the squirrel and the lion

In [ ]:
# prompts = ["A painting of a squirrel eating a burger",
#            "A painting of a lion eating a burger"]
# lb = LocalBlend(prompts, ("squirrel", "lion"))
# controller = AttentionReplace(prompts, NUM_DIFFUSION_STEPS,
#                               cross_replace_steps={"default_": 1., "lion": .4},
#                               self_replace_steps=0.4, local_blend=lb)
# _ = run_and_display(prompts, controller, latent=x_t, run_baseline=False)

In [ ]:
# prompts = ["A painting of a squirrel eating a burger",
#            "A painting of a squirrel eating a lasagne"]
# lb = LocalBlend(prompts, ("burger", "lasagne"))
# controller = AttentionReplace(prompts, NUM_DIFFUSION_STEPS,
#                               cross_replace_steps={"default_": 1., "lasagne": .2},
#                               self_replace_steps=0.4,
#                               local_blend=lb)
# _ = run_and_display(prompts, controller, latent=x_t, run_baseline=True)

## Refinement edit

In [ ]:
prompts = ["A blue SUV in the woods",
           "A blue  in the woods"]

lb = LocalBlend(prompts, ("SUV", "SUV"))
# equalizer = get_equalizer(prompts[1], ("watercolour",), (5,))
controller = AttentionRefine(prompts, NUM_DIFFUSION_STEPS,
                             cross_replace_steps=0.2, 
                             self_replace_steps=.4, local_blend=lb)
image_inv, _ = run_and_display_edit(prompts, controller, latent=ddim_latents[-1], latent_unt=ori_ddim_latents, guidance_scale=20, guidance_start_time = 0.2, run_baseline=False, generator=g_cpu)
# save_plot(image_inv[1], 'images/image6_3.png')

In [ ]:
prompts = ["a sitting cat .",
           "a sitting sculptural cat ."
        ]

lb = LocalBlend(prompts, ("cat", "cat"))
# equalizer = get_equalizer(prompts[1], ("sculptural",), (5,))
controller = AttentionRefine(prompts, NUM_DIFFUSION_STEPS,
                             cross_replace_steps=.2, 
                             self_replace_steps=.2, local_blend=lb)
image_inv, _ = run_and_display_edit(prompts, controller, latent=ddim_latents[-1], latent_unt=ori_ddim_latents, guidance_scale=14, guidance_start_time = 0.2, run_baseline=False, generator=g_cpu)
save_plot(image_inv[1], 'images/image1_3.png')

In [ ]:
prompts = ["A blue SUV in the autumn woods"] * 2

### pay 3 times more attention to the word "smiling"
lb = LocalBlend(prompts, ("woods", "woods"))
equalizer = get_equalizer(prompts[1], ("autumn",), (6,))
controller = AttentionReweight(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.5,
                               self_replace_steps=.6, local_blend=lb,
                               equalizer=equalizer)
image_inv, _ = run_and_display_edit(prompts, controller, latent=ddim_latents[-1], latent_unt=ori_ddim_latents, guidance_scale=6, guidance_start_time = 0.6, run_baseline=False, generator=g_cpu)
save_plot(image_inv[1], 'images/image6_2.png')

In [ ]:
prompts = ["a cat sitting next to a mirror",
           "watercolor painting of a cat sitting next to a mirror"
        ]
# lb = LocalBlend(prompts, ("cat", "cat"))
# equalizer = get_equalizer(prompts[1], ("watercolor",), (5,))
controller = AttentionRefine(prompts, NUM_DIFFUSION_STEPS,
                             cross_replace_steps=.8, 
                             self_replace_steps=.4)
image_inv, _ = run_and_display_edit(prompts, controller, latent=ddim_latents[-1], latent_unt=ori_ddim_latents, guidance_scale=12, guidance_start_time = 0.2, run_baseline=True, generator=g_cpu)

In [ ]:
prompts = ["a cat sitting next to a mirror",
           "a sleeping cat sitting next to a mirror"
        ]
lb = LocalBlend(prompts, ("cat", "cat"))
controller = AttentionRefine(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8,
                               self_replace_steps=.4, local_blend=lb)
image_inv, _ = run_and_display_edit(prompts, controller, latent=ddim_latents[-1], latent_unt=ori_ddim_latents, guidance_scale=20, guidance_start_time = 0.3, run_baseline=True, generator=g_cpu)

In [ ]:
g_cpu = torch.Generator().manual_seed(8888)
prompts = ["A painting of a squirrel eating a burger"]
controller = AttentionStore()
image, x_t = run_and_display(prompts, controller, latent=None, run_baseline=False, generator=g_cpu)
show_cross_attention(controller, res=16, from_where=("up", "down"))

In [ ]:
# prompts = ["A painting of a squirrel eating a burger",
#            "A painting of a lion eating a burger"]
prompts = ["a sitting cat .",
           "a sitting dog ."]
lb = LocalBlend(prompts, ("cat", "dog"))
controller = AttentionReplace(prompts, NUM_DIFFUSION_STEPS,
                              cross_replace_steps={"default_": 1., "lion": .4},
                              self_replace_steps=0.4, local_blend=lb)
_ = run_and_display(prompts, controller, latent=x_t, run_baseline=False)

In [ ]:
prompts = ["a photo of a house on a mountain",
           "a photo of a house on a mountain at fall"]


controller = AttentionRefine(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8,
                             self_replace_steps=.4)
_ = run_and_display(prompts, controller, latent=x_t)

In [ ]:
prompts = ["a photo of a house on a mountain",
           "a photo of a house on a mountain at winter"]


controller = AttentionRefine(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8,
                             self_replace_steps=.4)
_ = run_and_display(prompts, controller, latent=x_t)

In [ ]:
prompts = ["soup",
           "pea soup"] 

lb = LocalBlend(prompts, ("soup", "soup"))

controller = AttentionRefine(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8,
                             self_replace_steps=.4,
                             local_blend=lb)
_ = run_and_display(prompts, controller, latent=x_t, run_baseline=False)

## Attention Re-Weighting

In [ ]:
prompts = ["a smiling bunny doll"] * 2

### pay 3 times more attention to the word "smiling"
equalizer = get_equalizer(prompts[1], ("smiling",), (5,))
controller = AttentionReweight(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8,
                               self_replace_steps=.4,
                               equalizer=equalizer)
_ = run_and_display(prompts, controller, latent=x_t, run_baseline=False)

In [ ]:
prompts = ["pink bear riding a bicycle"] * 2

### we don't wont pink bikes, only pink bear.
### we reduce the amount of pink but apply it locally on the bikes (attention re-weight + local mask )

### pay less attention to the word "pink"
equalizer = get_equalizer(prompts[1], ("pink",), (-1,))

### apply the edit on the bikes 
lb = LocalBlend(prompts, ("bicycle", "bicycle"))
controller = AttentionReweight(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8,
                               self_replace_steps=.4,
                               equalizer=equalizer,
                               local_blend=lb)
_ = run_and_display(prompts, controller, latent=x_t, run_baseline=False)

### Where are my croutons?
It might be useful to use Attention Re-Weighting with a previous edit method.

In [ ]:
prompts = ["soup",
           "pea soup with croutons"] 
lb = LocalBlend(prompts, ("soup", "soup"))
controller = AttentionRefine(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8,
                             self_replace_steps=.4, local_blend=lb)
_ = run_and_display(prompts, controller, latent=x_t, run_baseline=False)

Now, with more attetnion to `"croutons"`

In [ ]:
prompts = ["soup",
           "pea soup with croutons"] 


lb = LocalBlend(prompts, ("soup", "soup"))
controller_a = AttentionRefine(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8, 
                               self_replace_steps=.4, local_blend=lb)

### pay 3 times more attention to the word "croutons"
equalizer = get_equalizer(prompts[1], ("croutons",), (3,))
controller = AttentionReweight(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8,
                               self_replace_steps=.4, equalizer=equalizer, local_blend=lb,
                               controller=controller_a)
_ = run_and_display(prompts, controller, latent=x_t, run_baseline=False)

In [ ]:
prompts = ["potatos",
           "fried potatos"] 
lb = LocalBlend(prompts, ("potatos", "potatos"))
controller = AttentionRefine(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8,
                             self_replace_steps=.4, local_blend=lb)
_ = run_and_display(prompts, controller, latent=x_t, run_baseline=False)

In [ ]:
prompts = ["potatos",
           "fried potatos"] 
lb = LocalBlend(prompts, ("potatos", "potatos"))
controller = AttentionRefine(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8, 
                             self_replace_steps=.4, local_blend=lb)

### pay 10 times more attention to the word "fried"
equalizer = get_equalizer(prompts[1], ("fried",), (10,))
controller = AttentionReweight(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.8,
                               self_replace_steps=.4, equalizer=equalizer, local_blend=lb,
                               controller=controller_a)
_ = run_and_display(prompts, controller, latent=x_t, run_baseline=False)